Bonus Lecture #1: Doing Data Science with <a href="https://plot.ly/">plotly</a>
=================

### Exploring the US election

In this lecture, we are going to explore some ways of visualizing data using plotly, a very cool visualization api which allows you to create professional-looking visualizations. We will do that in the context of Presidential election! We are going to combine data from various sources, just as you would do in real life data science! The data that we are using is a processed version of those found in the below sources.

**US Election results** can be found <a href="http://www.stat.berkeley.edu/users/nolan/data/voteProject/2016_US_County_Level_Presidential_Results.csv">here</a>.

**Median Income data by state** can be found here <a href="https://www.census.gov/prod/2012pubs/acsbr11-02.pdf">here</a>.

**Gini coefficient** data lives <a href="http://www.census.gov/topics/income-poverty/income-inequality.html">here</a>.

In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
from datascience import *
import numpy as np
import math
import requests
import re
import json

py.sign_in('data8', 'qOP6GPnJlLfnS2TnwDvs')

## Let's take a look at Michigan

In [6]:
#Let's find the voting and census data for all the Michigan counties
election_16 = Table.read_table("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/merged_election_data.csv")
mi_voting_data = election_16.where(election_16.column("State") == "michigan")
mi_voting_data

Unnamed: 0,County Name,State,ObamaVotes.12,RomneyVotes.12,votes_dem.16,votes_gop.16,total_votes.16,per_dem.16,per_gop.16,diff.16,per_point_diff.16,bushVote.04,kerryVote.04,Total.Precincts.08,Precincts.Reporting.08,Obama.08,McCain.08,Other.08,name,x_coord,y_coord,state_abbr,total_population,white_population,white_population_error,black_population,black_population_error,total_households,average_household_size,average_family_size,fertility,enrolled_hs,enrolled_higher_ed,edu_less_than_9th,edu_hs_no_diploma,edu_hs_diploma,edu_college_no_degree,edu_college_bachelors,edu_college_graduate_professional,in_labor_force_yes,in_labor_force_no,public_transportation_to_work,working_at_home,occupation_management_business_science_arts,income_less_than_10000,income_10000_to_24999,income_25000_to_49999,income_50000_to_99999,income_100000_to_149999,income_150000_to_199999,income_more_than_200000
39,alcona,michigan,2462,3571,1732,4201,6179,0.280304,0.679883,1037,1111,3590,2871,12,12,2896,3404,84,Alcona,-83.2729,44.7122,MI,11238,10998,257,nan,nan,4608,2.43,2.94,77,559,309,337,1064,3758,2012,681,424,4137,5643,7,313,809,363,1229,1520,1194,202,202,52
45,alger,michigan,2212,2329,1663,2585,4513,0.368491,0.57279,2648,372,2316,2395,14,14,2472,2188,90,Alger,-86.4842,47.1604,MI,9604,8366,498,789,187,3688,2.24,2.73,69,545,255,358,661,3196,1382,917,330,3788,4376,47,224,915,296,800,1194,1062,237,237,47
47,allegan,michigan,19594,29023,17932,33812,55153,0.325132,0.613058,515,673,32952,19227,48,48,23526,29526,1025,Allegan,-86.25,42.565,MI,111385,103163,717,1433,317,42078,2.62,3.04,1725,6823,5093,3033,5061,28472,16513,9689,4460,56049,29190,92,1946,13907,2423,6495,12028,15327,4198,4198,762
59,alpena,michigan,6549,7298,4877,9090,14688,0.33204,0.618873,1674,666,7665,7406,18,18,7705,7125,255,Alpena,-83.1976,45.037,MI,29958,29241,819,nan,nan,13357,2.21,2.7,382,1644,1645,829,1822,7450,5409,1938,1370,14353,10115,41,665,3941,1119,3524,3876,3528,977,977,118
78,antrim,michigan,5107,7917,4448,8469,13572,0.327734,0.624005,1623,704,8379,5072,16,16,6079,7506,267,Antrim,-85.1769,45.006,MI,23975,23137,267,nan,nan,10043,2.33,2.75,250,1287,741,398,1419,6473,3863,2413,1631,11084,8458,62,654,2706,669,2141,3043,3008,807,807,192
87,arenac,michigan,3669,4054,2238,4704,7457,0.300121,0.630817,1034,830,4071,4076,20,20,4155,3807,166,Arenac,-83.7524,44.0455,MI,16487,15781,679,nan,nan,6686,2.33,2.83,112,951,521,672,1572,5041,2569,843,437,7091,6591,69,325,1654,570,1626,2241,1780,318,318,59
134,baraga,michigan,1574,1866,1156,2158,3486,0.331612,0.619048,2,671,1977,1660,8,8,1725,1846,73,Baraga,-88.342,46.7154,MI,8882,6671,714,599,299,3336,2.39,2.94,87,480,238,482,678,2799,1342,539,203,3841,3509,27,81,1037,326,620,1098,1019,201,201,38
144,barry,michigan,11496,16651,9109,19197,30265,0.300975,0.634297,10,840,18638,11312,25,25,13449,16431,685,Barry,-85.311,42.5978,MI,59576,58075,1117,nan,nan,22843,2.6,3.01,636,3644,3007,928,2883,16028,9836,4152,2494,30505,15866,14,762,7014,955,3586,6372,8510,2409,2409,416
156,bay,michigan,26797,23735,21641,28327,52932,0.408845,0.535158,2233,100,25443,31041,89,89,32589,23794,1044,Bay,-83.9427,43.7208,MI,108156,101781,650,1582,440,44345,2.41,2.91,1271,6056,6707,2739,6679,26758,17321,9178,4226,54158,32537,283,1573,13927,3528,7931,13545,13709,4022,4022,634
194,benzie,michigan,4683,5071,4108,5539,10221,0.401918,0.541923,393,149,5283,4383,13,13,5461,4687,171,Benzie,-86.2478,44.6304,MI,17705,17059,1009,nan,nan,7366,2.35,2.83,170,1032,753,232,1060,4302,2748,1922,1292,8754,5599,34,473,2441,372,1314,2510,2419,547,547,94


In [3]:
# Add a new column indicating % unemployed and % of GOP vote per county
mi_voting_data = mi_voting_data.with_column("perc_labor_force", mi_voting_data.column("in_labor_force_yes")/(mi_voting_data.column("in_labor_force_yes") + mi_voting_data.column("in_labor_force_no")))
mi_voting_data = mi_voting_data.with_column("perc_gop_vote", mi_voting_data.column("votes_gop.16")/(mi_voting_data.column("votes_gop.16") + mi_voting_data.column("votes_dem.16")))

In [4]:
# Break the counties into 2 tables: one for Democrat-voting counties and the other for Republic-voting counties
gop_counties = mi_voting_data.where(mi_voting_data.column("perc_gop_vote") > 0.5)
dem_counties = mi_voting_data.where(mi_voting_data.column("perc_gop_vote") < 0.5)

In [5]:
trace_gop = go.Scatter(
    name = "Republican",
    x = gop_counties.column("perc_labor_force"),
    y = gop_counties.column("perc_gop_vote"),
    text = gop_counties.column("County Name"),
    mode = 'markers'
)

trace_dem = go.Scatter(
    name = "Democrat",
    x = dem_counties.column("perc_labor_force"),
    y = dem_counties.column("perc_gop_vote"),
    text = dem_counties.column("County Name"),
    mode = 'markers'
)

data = [trace_gop, trace_dem]

layout = go.Layout(
    title='Percent Unemployed versus GOP Votes in Michigan',
    xaxis = dict(
        title = "% Employed",
    ),
    yaxis = dict(
        title = "% Votes for Republicans",
    )
)

fig = go.Figure(data = data, layout = layout)

py.iplot(fig, filename='employment-vs-votes')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~data8/0 or inside your plot.ly account where it is named 'employment-vs-votes'


## Election Results Overview 

In [ ]:
#First off let's read in our data and take a quick look
#One easy way to read in your data from a csv file is using the datascience package's table
#You will learn more about the datascience package next week!
df = Table.read_table("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/election_df_2016.csv").drop("code")
df.show(5)

In [ ]:
#The first step is to extract the data we are interested in from our table
#all the statements below return the corresponding table columns in the form of a numpy array.
state_abbreviations = df.column("state_abbr").astype(str)
text_16 = df.column("text").astype(str)
perc_difference_16 = df.column("perc_diff").astype(str)
votes_dem_16 = df.column("votes_dem.16").astype(int)
votes_gop_16 = df.column("votes_gop.16").astype(int)
df_ = Table.read_table("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/election_df_2012.csv")
perc_difference_12 = df_.column("perc_diff").astype(str)

In [ ]:
#Determine the color scale that we will be using
scl = [[0, 'rgb(255, 0, 0)'],[0.25, 'rgb(255, 122, 122)'],[0.4, "rgb(247, 133, 133)"],[0.45, 'rgb(255, 153, 153)'],
       [0.5, 'rgb(255, 255, 255)'],[0.55, 'rgb(160, 160, 255)'],[0.6, 'rgb(133, 177, 247)'], [0.75, 'rgb(122,122,255)'],[1, 'rgb(0, 0, 255)']]

#first off we have to put the data in the right format
data = [ dict(
        type='choropleth',
        autocolorscale = False,
        locations = state_abbreviations, #the state abbreviations go here
        z = perc_difference_16.astype(float),
        locationmode = 'USA-states', #determine the type of map you want to make
        text = text_16, #this is the array containing the information we will see when we hover over our states
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Percentage Difference"), #Title of our map
            colorscale = scl, #set the colorscale we created
            zmin = -0.5,
            zmax = 0.5
        ) ]

#make the layout that contains information about the map
layout = dict(
        title = '2016 Election Results',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [ ]:
#Now I want to calculate the percentage that each party got on each state - numpy math
perc_dem = votes_dem_16/(votes_dem_16+votes_gop_16)
perc_gop = votes_gop_16/(votes_dem_16+votes_gop_16)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    y=state_abbreviations,
    x=perc_dem,
    name='Democrats',
    orientation = 'h',
    marker = dict(
        color = 'rgba(169, 228, 232, 0.4)',
        line = dict(
            color = 'rgba(169, 228, 232, 1.0)',
            width = 5)
    )
)
trace2 = go.Bar(
    y=state_abbreviations,
    x=perc_gop,
    name='Republicans',
    orientation = 'h',
    marker = dict(
        color = 'rgba(224, 92, 45, 0.6)',
        line = dict(
            color = 'rgba(224, 92, 45, 1.0)',
            width = 5)
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

## In how many states did each party improve their performance?

This election was admittedly one of the most polarized of the recent years. So one question to ask is how did that affect the electoral map?

In [ ]:
#explore in how many states republicans and democrats improved their percentage
perc_difference_12 = perc_difference_12.astype(float)
perc_difference_16 = perc_difference_16.astype(float)
diff = perc_difference_16-perc_difference_12
diff

In [ ]:
#Get rid of the nan value
diff = diff[~np.isnan(diff)]
dems_improved = np.count_nonzero(diff>0)
republicans_improved = np.count_nonzero(diff<0)

In [ ]:
#Bar chart
data = [go.Bar(
            x=['Democrats improved', 'Republicans improved'],
            y=[dems_improved, republicans_improved]
    )]
layout = go.Layout(
    title="In how many states did the two parties improve?"
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

In [ ]:
#This is very high level. Let's try to see how many states republicans actually converted compared to the 2012 election
gop_12 = perc_difference_12<0 #won by the gop in 2012
gop_16 = perc_difference_16<0 #won by the gop in 2016
lst = list(zip(gop_12, gop_16))
lst

In [ ]:
from collections import Counter
counts = dict(Counter(lst))
counts

In [ ]:
counts["democrat_to_democrat"] = counts.pop((False, False))
counts["democrat_to_republican"] = counts.pop((False, True))
counts["republican_to_republican"] = counts.pop((True, True))
counts["republican_to_democrat"] = 0
counts

In [ ]:
#Get all items from a dictionary in alphabetical order of the keys
sorted_list = sorted(counts.items(), key=lambda x: x[0])
sorted_list

In [ ]:
trace0 = go.Bar(
    x=['Democrat to Democrat', 'Democrat to Republican', 'Republican to Democrat',
       'Republican to Republican'],
    y=[val for key, val in sorted_list], #I want the keys in sorted order
    marker=dict(
        color=['rgba(204,204,204,1)','rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(222,45,38,0.8)']),
)

data = [trace0]
layout = go.Layout(
    title='How many states did the two parties manage to convert?',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='color-bar')

## Voting and income inequality

Now we will take a look at how income and income inequality relate to the voting patterns.

In [ ]:
import csv
#we will need that dictionary to put our data in a similar format
with open('state_abbreviations.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    state_abbr_dict = dict(reader)
state_abbr_dict['district of columbia'] = "DC"

In [ ]:
import pprint
with open('gini_states_data.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    gini_data = dict(reader)
    
#Put the data in the right format
for key in gini_data.keys():
    gini_data[key] = eval(gini_data[key])
    
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(gini_data)

In [ ]:
#replace states by state abbreaviations
states_encoded = []
for state in gini_data['State']:
    state = state.lower() #convert to lowercase
    if state in state_abbr_dict.keys():
        states_encoded.append(state_abbr_dict[state])
gini_data['State'] = states_encoded

In [ ]:
gini = list(zip(gini_data['Year'], gini_data['State'], gini_data['Gini']))
#keep data from 2010
gini_2010 = [i for i in gini if i[0]==2010]
len(gini_2010)

In [ ]:
sorted(gini_2010, key = lambda x:x[1])

In [ ]:
gini_2010.remove((2010, 'SD', 0.655813515))
len(gini_2010)

In [ ]:
gini_2010 = dict([(i[1], i[2]) for i in gini_2010])
gini_2010

In [ ]:
#income data
income = Table.read_table("states_median_income.csv")
income.show(5)

In [ ]:
#format the income column
income.append_column("median_income",[int(i.replace(",", "")) for i in income.column("median_income")])
income.append_column("State", [state_abbr_dict[key] for key in income.column("State")])

In [ ]:
median_income = income.sort("State").column("median_income")
gini = [i[1] for i in sorted(list(gini_2010.items()), key=lambda x:x[0])]

In [ ]:
electoral = Table.read_table("electoral_votes.csv").select(["State", "electoral_votes"]).take(np.arange(51))
electoral['State'] = electoral.apply(lambda x: state_abbr_dict[x.lower()], "State")
electoral_votes = electoral.sort("State").column("electoral_votes")

In [ ]:
hover_text = []
bubble_size = []
for i in range(len(state_abbreviations)): 
    hover_text.append(('State: {state}<br>'+
                          'Electoral Votes: {evotes}<br>'+
                          'Median income: {income}<br>'+
                          'Gini coefficient: {gini}<br>').format(state=state_abbreviations[i],
                                                evotes=electoral_votes[i],
                                                income=median_income[i],
                                                gini=gini[i]
                                                ))
    bubble_size.append(electoral_votes[i])

In [ ]:
democratic_states = perc_difference_16>0
republicans_states = perc_difference_16<0

In [ ]:
text = hover_text
size = bubble_size

trace0 = go.Scatter(
    x=median_income[democratic_states],
    y=np.array(gini)[democratic_states],
    mode='markers',
    name='Democratic',
    text=np.array(text)[democratic_states],
    marker=dict(
        symbol='circle',
        sizemode='diameter',
        sizeref=0.85,
        size=np.array(size)[democratic_states],
        line=dict(
            width=2
        )
    )
)
trace1 = go.Scatter(
    x=median_income[republicans_states],
    y=np.array(gini)[republicans_states],
    mode='markers',
    name='Republican',
    text=np.array(text)[republicans_states],
    marker=dict(
        sizemode='diameter',
        sizeref=0.85,
        size=np.array(size)[republicans_states],
        line=dict(
            width=2
        ), color = 'rgb(243, 0, 0)'
    )
)

data = [trace0, trace1]
layout = go.Layout(
    title = "Income inequality and the 2016 election",
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='2106')

## Bonus material - County Level Map for Michigan

In [13]:
#Let's import geo data that will allow us to map all the counties
url = 'http://catalog.civicdashboards.com/dataset/08b4daa7-8888-4e03-ad73-ae2ff3c47771/resource/b12bbaa4-14df-4293-8636-2bb80804678c/download/89e296fe1c5c410d83c2d97155366e24temp.geojson'
michigan_data = requests.get(url)
michigan_data = michigan_data.json()
michigan_data

{'features': [{'geometry': {'coordinates': [[[[-85.864155, 46.702662],
       [-85.864932, 46.504757999999995],
       [-86.364953, 46.505852],
       [-86.36526099999999, 46.331928999999995],
       [-86.615397, 46.333262999999995],
       [-86.615601, 46.158691],
       [-87.11608, 46.158944999999996],
       [-87.11663399999999, 46.494808],
       [-87.112245, 46.749925999999995],
       [-87.112695, 47.817997999999996],
       [-86.94062199999999, 47.749905],
       [-86.949226, 47.749905],
       [-86.75012, 47.67196],
       [-86.75012, 47.674910999999994],
       [-86.631321, 47.624919999999996],
       [-86.634344, 47.624919999999996],
       [-86.625125, 47.617767],
       [-86.625125, 47.620008],
       [-86.50013, 47.568324],
       [-86.25013899999999, 47.477903],
       [-86.25013899999999, 47.474447999999995],
       [-86.125141, 47.423468],
       [-86.125142, 47.416744],
       [-86.020032, 47.374962],
       [-86.027345, 47.374961],
       [-86.00014499999999, 47.36188

In [14]:
#Let's find all the county names and store their data in a dictionary
county_names = []
county_names_dict = {}

for county in michigan_data['features']:
    for m in range(len(county['properties']['name'])):
        if county['properties']['name'][m:m+6] == 'County':
            county_names.append(county['properties']['name'][0:m-1].lower())
            county_names_dict[county['properties']['name'][0:m-1]] = county['properties']['name']
            
print(county_names)

['alger', 'cheboygan', 'barry', 'kalamazoo', 'menominee', 'grand traverse', 'mackinac', 'kalkaska', 'ottawa', 'roscommon', 'saginaw', 'ogemaw', 'tuscola', 'mason', 'ingham', 'ontonagon', 'branch', 'montmorency', 'osceola', 'wexford', 'alcona', 'allegan', 'huron', 'hillsdale', 'dickinson', 'chippewa', 'leelanau', 'clare', 'baraga', 'alpena', 'gratiot', 'wayne', 'oscoda', 'bay', 'muskegon', 'keweenaw', 'houghton', 'cass', 'crawford', 'newaygo', 'midland', 'gladwin', 'washtenaw', 'mecosta', 'luce', 'oceana', 'jackson', 'montcalm', 'sanilac', 'shiawassee', 'eaton', 'st. joseph', 'schoolcraft', 'st. clair', 'isabella', 'benzie', 'kent', 'emmet', 'monroe', 'gogebic', 'manistee', 'antrim', 'iosco', 'lake', 'livingston', 'delta', 'van buren', 'berrien', 'missaukee', 'macomb', 'iron', 'lenawee', 'ionia', 'arenac', 'calhoun', 'clinton', 'presque isle', 'charlevoix', 'otsego', 'genesee', 'lapeer', 'marquette', 'oakland']


In [8]:
#Let's classify counties as Democrat or Republican
blue_counties_2016 = []
red_counties_2016 = []

for k, county in enumerate(county_names):
    county_cleaned = re.sub("\.", "", county)
    county_cleaned = re.sub(" ", "", county_cleaned)
    county_cleaned = re.sub(".*-", "", county_cleaned)
    row = mi_voting_data.where(mi_voting_data[1] == county_cleaned)
    if (row.num_rows != 1):
        print("Error: " + county_cleaned)
    else:
        if (row["votes_dem.16"].item() > row["votes_gop.16"].item()):
            blue_counties_2016.append(michigan_data['features'][k])
        else:
            red_counties_2016.append(michigan_data['features'][k])
print("Dem Counties: \n " + str(blue_counties_2016))
print("GOP Counties: \n " + str(red_counties_2016))

Dem Counties: 
 [{'type': 'Feature', 'properties': {'name': 'Kalamazoo County, MI', 'geoid': '05000US26077'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-85.418099, 42.420193999999995], [-85.298879, 42.419849], [-85.299557, 42.419340999999996], [-85.297088, 42.249026], [-85.29362599999999, 42.071553], [-85.76294299999999, 42.069327], [-85.766066, 42.244693], [-85.764352, 42.420669], [-85.53249799999999, 42.421521999999996], [-85.418099, 42.420193999999995]]]]}}, {'type': 'Feature', 'properties': {'name': 'Ingham County, MI', 'geoid': '05000US26065'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-84.60186399999999, 42.769873], [-84.36366, 42.769971], [-84.363659, 42.775777999999995], [-84.158189, 42.776638999999996], [-84.151354, 42.702245], [-84.14174799999999, 42.547264999999996], [-84.14060099999999, 42.508393999999996], [-84.14071299999999, 42.424611], [-84.36480499999999, 42.424693999999995], [-84.364783, 42.422146999999995], [-84.600403, 42.421983999999995]

In [11]:
#Now we write our data to a JSON file, which will allow us to import it back to Plotly.
red_data_2016 = {"type": "FeatureCollection"}
red_data_2016['features'] = red_counties_2016

blue_data_2016 = {"type": "FeatureCollection"}
blue_data_2016['features'] = blue_counties_2016

with open('florida-red-data-2016.json', 'w') as f:
    f.write(json.dumps(red_data_2016))
with open('florida-blue-data-2016.json', 'w') as f:
    f.write(json.dumps(blue_data_2016))


In [12]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

mapbox_access_token = 'pk.eyJ1IjoiYWRuYW5oZW1hbmkiLCJhIjoiY2l5N3RnY3M4MDAxZTJxb3Y3anlsbnJ0ZSJ9.REcsaI8vU6ylkrpkjuMtqA'

data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
    )
])
layout = graph_objs.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/adnanhemani/data8-assets/master/michigan-red-data-2016.json',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/adnanhemani/data8-assets/master/michigan-blue-data-2016.json',
                type = 'fill',
                color = 'rgba(40,0,113,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=44.5,
            lon=-84.8
        ),
        pitch=0,
        zoom=4.5,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')
